# Sentiment Anlysis

In [1]:
# !pip install nltk
# !pip install re
# !pip install kaggle
# !pip install gensim
import pandas as pd
import numpy as np
import os
import re
import spacy
import gensim.downloader
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Conv1D, Dense, MaxPooling1D, GlobalMaxPooling1D, Flatten, Input, Dropout
from keras.layers.embeddings import Embedding
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
PATH = r'D:\uchi\2021Fall\PPHA30536_Data and Programming for Public Policy II\final-project-tianling-luo'

## Prepare dataset for training

In [3]:
"""
(Can skip this step if one do not want to use kaggle API)
If using kaggle API, please uncomment the following 4 lines
"""
# import kaggle
# kaggle.api.authenticate()
# kaggle.api.dataset_download_files('lakshmi25npathi/imdb-dataset-of-50k-movie-reviews', 
#                                   path=os.path.join(PATH, 'Data'), unzip=True)

In [4]:
# read data from csv file
def read_csv(fname):
    df = pd.read_csv(os.path.join(PATH, 'Data', fname))
    return df

In [5]:
# label the reviews by 1 and 0
def label_sentiment(df):
    df['label'] = 0
    for ind in df.index:
        if df.loc[ind, ['sentiment']].values[0] == 'positive':
            df.loc[ind, ['label']] = 1
        if df.loc[ind, ['sentiment']].values[0] == 'negative':
            df.loc[ind, ['label']] = 0
    return df

In [6]:
# remove undesired signs and characters
def clean_text(sen):
    # remove html tags
    re.sub(r'<[^>]+>', ' ', sen)
    # remove punctuations and numbers
    sen = re.sub('[^a-zA-Z]', ' ', sen)
    # single character removal
    sen = re.sub(r'\s+[a-zA-Z]\s+', ' ', sen)
    # remove multiple spaces
    sen = re.sub(r'\s+', ' ', sen)   
    return sen

In [7]:
# change capital letters to lower ones
def lower_token(token): 
    lower_token = []
    for word in token:
        lower_token.append(word.lower())
    return lower_token

In [8]:
# remove stopwords
def remove_stopwords(token): 
    stoplist = stopwords.words('english')
    filtered_token = []
    for word in token:
        if word not in stoplist:
            filtered_token.append(word)
    return filtered_token

In [9]:
def overall_clean_text(column_name, df):
    cleaned_column = column_name + '_clean'
    final_column = column_name + '_final'
    
    df[cleaned_column] = df[column_name].apply(lambda sen: clean_text(sen))
    # transfer sentences to tokens
    tokens = [word_tokenize(sen) for sen in df[cleaned_column]]
    # change tokens to lower letters
    lower_tokens = [lower_token(token) for token in tokens] 
    # remove stopwords
    filtered_tokens = [remove_stopwords(token) for token in lower_tokens]
    # combine results
    result = [' '.join(token) for token in filtered_tokens] 
    df[final_column] = result
    df['tokens'] = filtered_tokens
    return df

In [10]:
# If you fail to download the file from kaggle, you can directly use this one.
movie_reviews = read_csv("IMDB Dataset.csv")
# label the reviews by 0 and 1
movie_reviews = label_sentiment(movie_reviews)

In [11]:
# clean the texts
movie_reviews = overall_clean_text('review', movie_reviews)
movie_reviews = movie_reviews[['review_final', 'tokens', 'label']]

## Building up CNN model

In [12]:
# get x and y data for training and testing
def get_x_y(t_x, t_data, maxlen):
    t_sequences = tokenizer.texts_to_sequences(t_x.tolist())
    x = pad_sequences(t_sequences, maxlen=maxlen)
    y = t_data['label'].values
    return x, y

In [13]:
# function for embeddings
def embeddings(tokens_list, word2vec, dim=300):
    if len(tokens_list)<1:
        return np.zeros(dim)   
    vec = [word2vec[word] if word in word2vec else np.random.rand(dim) for word in tokens_list]
    average = len(vec) / np.sum(vec, axis=0)
    return average

In [14]:
# CNN architechture
def cnn_construct(vocab_size, dim, embedding_matrix, maxlen):
    input_data = Input(shape=(maxlen,))
    embedding_layer = Embedding(vocab_size, 
                                dim, 
                                weights=[embedding_matrix], 
                                input_length=maxlen, 
                                trainable=False)
    embedding = embedding_layer(input_data)
    conv1 = Conv1D(filters=64, kernel_size=5, activation='relu')(embedding)
    pool1 = MaxPooling1D(2)(conv1)
    conv2 = Conv1D(filters=64, kernel_size=5, activation='relu')(pool1)
    pool2 = GlobalMaxPooling1D()(conv2)
    dropout = Dropout(0.5)(pool2)
    dense1 = Dense(64, activation='relu')(dropout)
    pred = Dense(1, activation='sigmoid')(dense1)
    
    model = Model(input_data, pred)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [15]:
# split train and test data
training_data, testing_data = train_test_split(movie_reviews, test_size=0.2)
training_x = training_data['review_final']
testing_x = testing_data['review_final']

In [16]:
"""
NOTICE: This step may take a bit long since the vector file is large.
If the storage is not enough for this download, you might directly use the result in sentiment.xlsx data.
"""
word2vec = gensim.downloader.load('word2vec-google-news-300')

In [17]:
training_embeddings = list(training_data['tokens'].apply(lambda x: embeddings(x, word2vec)))

In [18]:
# set variables
maxlen = 100
dim = 300

training_words = [word for tokens in training_data["tokens"] for word in tokens]
training_wordlist = sorted(list(set(training_words)))

In [19]:
# tokenize and padding
tokenizer = Tokenizer(num_words=len(training_wordlist), lower=True, char_level=False)
tokenizer.fit_on_texts(training_x.tolist())
train_x, train_y = get_x_y(training_x, training_data, maxlen)
test_x, test_y = get_x_y(testing_x, testing_data, maxlen)

word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

In [20]:
# get embedding matrix
embedding_matrix = np.zeros((vocab_size, dim))
for word, index in word_index.items():
    if word in word2vec:
        embedding_matrix[index,:] = word2vec[word]
    else:
        np.random.rand(dim)

In [21]:
# construct CNN model
model = cnn_construct(vocab_size, dim, embedding_matrix, maxlen)
callbacks_list = [EarlyStopping(monitor='var_loss', min_delta=0.01, patience=4, verbose=1)]

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding (Embedding)       (None, 100, 300)          27153300  
                                                                 
 conv1d (Conv1D)             (None, 96, 64)            96064     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 48, 64)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 44, 64)            20544     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                              

In [22]:
# train model
epochs = 16
batch_size = 64
hist = model.fit(train_x, train_y, epochs=epochs, verbose=1, validation_split=0.15, shuffle=True, batch_size=batch_size)

Epoch 1/16
532/532 [==============================] - 48s 82ms/step - loss: 0.4447 - acc: 0.7876 - val_loss: 0.3996 - val_acc: 0.8197
Epoch 2/16
532/532 [==============================] - 37s 70ms/step - loss: 0.3349 - acc: 0.8591 - val_loss: 0.3304 - val_acc: 0.8567
Epoch 3/16
532/532 [==============================] - 36s 68ms/step - loss: 0.2980 - acc: 0.8769 - val_loss: 0.3600 - val_acc: 0.8453
Epoch 4/16
532/532 [==============================] - 36s 68ms/step - loss: 0.2561 - acc: 0.8961 - val_loss: 0.3337 - val_acc: 0.8595
Epoch 5/16
532/532 [==============================] - 36s 67ms/step - loss: 0.2160 - acc: 0.9135 - val_loss: 0.3707 - val_acc: 0.8477
Epoch 6/16
532/532 [==============================] - 37s 69ms/step - loss: 0.1858 - acc: 0.9262 - val_loss: 0.3519 - val_acc: 0.8593
Epoch 7/16
532/532 [==============================] - 32s 60ms/step - loss: 0.1527 - acc: 0.9407 - val_loss: 0.4118 - val_acc: 0.8608
Epoch 8/16
532/532 [==============================] - 32s 60ms

In [23]:
# test model
evaluation = model.evaluate(test_x, test_y, verbose=1)
accuracy = evaluation[1]
print("Accuracy is: ", accuracy)

313/313 [==============================] - 4s 12ms/step - loss: 0.7096 - acc: 0.8472
Accuracy is:  0.8471999764442444


## Predict on the SOTU texts

In [24]:
# transfer texts to lists of sentences
def txt_to_sents(txts):
    text_sents = []
    text_sents_num = []
    for num in range(len(txts)):
        filename = os.path.join(PATH, 'Data\SOTU texts', txts[num])
        text_file = open(filename,'r',encoding='UTF-8-sig', errors='ignore')
        text = text_file.read()
        text = text.replace('\n','')
        nlp = spacy.load("en_core_web_sm")
        doc = nlp(text)
        sents = list(doc.sents)
        text_sents.append(sents)
        text_sents_num.append(len(sents))
    return text_sents, text_sents_num

In [25]:
# transfer text sentences into list of DataFrames
def df_list(num_txts, text_sents):
    df_text = []
    for num in range(num_txts):
        text_sents[num]  = [sen.text for sen in text_sents[num]]
        df = pd.DataFrame(text_sents[num])
        df.columns = ['address']
        df_text.append(df)
    return df_text

In [26]:
# prepare address texts for CNN prediction
def prep_address(df, maxlen):
    df = overall_clean_text('address', df)
    address = df['address_final']
    address_sequences = tokenizer.texts_to_sequences(address.tolist())
    address_x = pad_sequences(address_sequences, maxlen=maxlen)
    return address_x

In [27]:
# get predictions through trained CNN
def get_predictions(address_x, batch_size):
    # predict sentiments
    predictions = model.predict(address_x, batch_size=batch_size, verbose=1)
    prediction_labels = []
    for p in predictions:
        if p[0] < 0.5:
            prediction_labels.append(0)
        else:
            prediction_labels.append(1)
    return prediction_labels

In [28]:
# texts of sotu address
txts = os.listdir(PATH + '\Data\SOTU texts')
num_txts = len(txts)

In [29]:
# break the whole text into sentences
text_sents, text_sents_num = txt_to_sents(txts)
# transfer each text to a DataFrame and get a list of DataFrames
df_text = df_list(num_txts, text_sents)

In [30]:
scores = []
for df in df_text:
    # prepare address texts
    address_x = prep_address(df, maxlen)
    # get predictions
    prediction_labels = get_predictions(address_x, 1024)
    # get a sentiment score of the whole address
    score = sum(prediction_labels) / len(prediction_labels)
    scores.append(score)

1/1 [==============================] - 0s 168ms/step


In [31]:
# scores

[0.7534246575342466,
 0.762589928057554,
 0.8846153846153846,
 0.8349514563106796,
 0.783625730994152,
 0.7757575757575758,
 0.8016528925619835,
 0.8783783783783784,
 0.9122807017543859,
 0.9285714285714286,
 0.8537414965986394,
 0.8290909090909091,
 0.8415300546448088,
 0.9102564102564102,
 0.9053497942386831,
 0.8711864406779661,
 0.8542274052478134,
 0.8794326241134752,
 0.8571428571428571,
 0.9042553191489362,
 0.8685258964143426,
 0.8249027237354085,
 0.7952755905511811,
 0.785,
 0.8759398496240601,
 0.8356164383561644,
 0.7890625,
 0.9105058365758755,
 0.8991935483870968,
 0.8911174785100286,
 0.8899082568807339,
 0.8697916666666666,
 0.8994708994708994,
 0.8918918918918919,
 0.9085365853658537,
 0.8638743455497382,
 0.8333333333333334,
 0.8629629629629629,
 0.926605504587156,
 0.896414342629482,
 0.8805031446540881,
 0.8597560975609756,
 0.8392156862745098,
 0.851145038167939,
 0.8835616438356164,
 0.8495575221238938,
 0.8757062146892656,
 0.8952380952380953,
 0.8571428571428571

## Save to excel file

In [32]:
# read data from excel file Allen
def read_excel(fname, sname, header):
    df = pd.read_excel(os.path.join(PATH, 'Data', fname), header=header, sheet_name=sname)
    return df

In [33]:
# get the dates for sotu and save to excel
sotu_date = read_excel('Data_Daily_1.1-2.4.xlsx', 'address', [0])
sotu_date['sentiment'] = scores
sotu_date = sotu_date[['Date', 'sentiment']]
sotu_date.to_excel(os.path.join(PATH, 'Data', 'sentiment.xlsx'))